In [127]:
import pydicom
import glob
import pathlib
import shutil

In [3]:
dcm = pydicom.read_file('/home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/MR1.3.12.2.1107.5.2.43.166029.2017091810002626445588484.dcm')
dcm_rt = pydicom.read_file('/home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/RTSTRUCT1.2.246.352.222.400.3853717121.452.1651730871.452.dcm',force=True)


In [4]:
print(dcm)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 198
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.43.166029.2017091810002626445588484
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.3
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_363'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'NORM', 'DIS2D']
(0008, 0012) Instance Creation Date              DA: '20170918'
(0008, 0013) Instance Creation Time              TM: '100027.584000'
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0

In [9]:
dcm["SeriesInstanceUID"]

(0020, 000e) Series Instance UID                 UI: 1.3.12.2.1107.5.2.43.166029.2017091810002494539488091.0.0.0

In [16]:

dcm_rt["SeriesInstanceUID"].value

'1.2.246.352.222.400.3853717121.452.1651730871.453'

In [11]:
dcm_rt[(0x0020,0x0011)]

(0020, 0011) Series Number                       IS: '1'

In [25]:
dcm_rt[(0x0020,0x0052)]
dcm_rt[(0x0020,0x0052)].value

(0020, 0052) Frame of Reference UID              UI: 1.3.12.2.1107.5.2.43.166029.2.20170918093406152.0.0.0

In [ ]:
# Match up the FoR UID with the images - if dcm_rt doesnt match up with any of them, look at the date and will need to overwrite the FoR UID (if this is incorrecy) with the correct date
# can use date to match them up, if the linkage between the FoR UID
# can loop through and find one that matches

# loop througuh all dicoms, look at study date, put in new folder

#dicom crawler -- https://github.com/pyplati/platipy/blob/master/platipy/dicom/io/crawl.py (look for line 350ish down)

In [139]:
input_folder='/home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004'
patient_id = '04'

mri_files = glob.glob(f'{input_folder}/MR*')
rt_files = glob.glob(f'{input_folder}/RTSTRUCT*')

# print(mri_files[0][83:])
# print(len('/home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/'))

for mri_file in mri_files:
    dcm = pydicom.read_file(mri_file)
    UI = dcm[(0x0020, 0x0052)].value
    mri_study_date = dcm[(0x0008,0x0020)].value
    # print(UI)
    # print(mri_study_date)

    for rt_file in rt_files:
        dcm_rt = pydicom.read_file(rt_file,force=True)
        rt_FoR_UI = dcm_rt[(0x0020,0x0052)].value
        study_date = dcm_rt[(0x0008,0x0020)].value
        # print('RT struct',rt_FoR_UI[43:47]) #30
        # print(study_date)
        if UI == rt_FoR_UI:
            print(mri_file, ' is the MRI file', rt_file, 'is the associated RT file')
            mri_ID = str(dcm[(0x0020, 0x0052)].value)
            # note - need to prevent duplicates... once a file has been copied need to remove from rt_files list/mri_files list? just make sure not to /delete/ any files

            path=f"/home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-0{patient_id}/{mri_ID}"
            pathlib.Path(path).mkdir(parents=True, exist_ok=True)
            file_name = mri_file[83:]
            original_mri = f'{input_folder}/{file_name}'
            target = f'{path}/{file_name}'

            rt_file_name=rt_file[83:]
            original_rt = f'{input_folder}/{rt_file_name}'
            target = f'{path}/{rt_file_name}'

            shutil.copyfile(original_mri, target)

            if rt_file in rt_files:
                rt_files.remove(rt_file)
            if mri_file in mri_files:
                mri_files.remove(mri_file)


/home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/MR1.3.12.2.1107.5.2.43.166029.2017091810081275908690975.dcm  is the MRI file /home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/RTSTRUCT1.2.246.352.222.400.3853717121.7176.1658304678.1004.dcm is the associated RT file
/home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/MR1.3.12.2.1107.5.2.43.166029.2017091810081275908690975.dcm  is the MRI file /home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/RTSTRUCT1.2.246.352.222.400.3853717121.452.1651730871.452.dcm is the associated RT file
/home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/MR1.3.12.2.1107.5.2.43.166029.2017091810081275908690975.dcm  is the MRI file /home/alicja/PET-LAB Code/PET-LAB/PETLAB_CONTOUR_PROCESSED/PROCESSED/set-2/WES-004/RTSTRUCT1.2.246.352.222.400.3853717121.452.1651730879.464.dcm is the associated RT file
/home

In [73]:
len('2017091810080')
for rt_file in rt_files:
    dcm_rt = pydicom.read_file(rt_file,force=True)
    rt_FoR_UI =dcm_rt[(0x0020,0x0052)].value #dcm_rt[0x0020, 0x000e].value
    study_date = dcm_rt[(0x0008,0x0020)].value
    # print('RT struct',rt_FoR_UI) #30
    # print(dcm_rt[0x0020, 0x000e].value)
    if UI == rt_FoR_UI:
        print(mri_file, ' is the MRI file', rt_file, 'is the associated RT file')
        # print(study_date)
    # elif mri_study_date == study_date:
    #     print(mri_file, ' is the MRI file', rt_file, 'is the associated RT file')

13

In [126]:
patient_id = '04'

In [138]:
rt_file[83:]

'RTSTRUCT1.2.246.352.222.400.3853717121.7176.1658304237.1002.dcm'

In [123]:
# I think the aim is to put these files into different folders - can this be based on their names in any way? or any info on the MRI file?


b'/'